<a href="https://colab.research.google.com/github/WalterPHD/Ai-Data/blob/main/U_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 1

In [6]:
!git clone https://github.com/zhixuhao/unet.git

fatal: destination path 'unet' already exists and is not an empty directory.


In [7]:
import os
os.chdir('/content/unet')

In [9]:
!sed -i 's/from keras.preprocessing.image/from tensorflow.keras.preprocessing.image/' /content/unet/data.py


In [11]:
!sed -i 's/Model(input =/Model(inputs=/g' /content/unet/model.py
!sed -i 's/output =/outputs=/g' /content/unet/model.py


In [13]:
!sed -i 's/Adam(lr =/Adam(learning_rate=/g' /content/unet/model.py


In [15]:
!sed -i "s/unet_membrane.hdf5/unet_membrane.keras/g" /content/unet/main.py


In [17]:
!sed -i 's/fit_generator/fit/g' /content/unet/main.py


In [19]:
!sed -i 's/predict_generator/predict/g' /content/unet/main.py


In [21]:
!sed -i 's/yield img/yield (img,)/g' /content/unet/data.py


In [23]:
# Replace the saveResult function in data.py to handle float images
!sed -i '/def saveResult(save_path, results):/,+6 s/imsave(os.path.join(save_path,"%d_predict.png"%i),img)/from skimage import img_as_ubyte\n        imsave(os.path.join(save_path,"%d_predict.png"%i), img_as_ubyte(img))/g' /content/unet/data.py


In [34]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [40]:
from data import testGenerator
import os

# Re-initialize the test generator
test_path = "data/membrane/test"
testGene = testGenerator(test_path)

# Get one batch from the test generator
sample_batch = next(testGene)

# Print the type and shape of the sample batch
print(type(sample_batch))
if isinstance(sample_batch, (list, tuple)):
    for i, item in enumerate(sample_batch):
        print(f"Item {i}: Type - {type(item)}, Shape - {item.shape if hasattr(item, 'shape') else 'N/A'}")
else:
    print(f"Shape - {sample_batch.shape if hasattr(sample_batch, 'shape') else 'N/A'}")

<class 'tuple'>
Item 0: Type - <class 'numpy.ndarray'>, Shape - (1, 256, 256, 1)


In [41]:

!sed -i 's/yield img/yield (img,)/g' /content/unet/data.py

In [30]:
import os
import numpy as np
from model import unet
from data import testGenerator, saveResult
from skimage import img_as_ubyte

# Load your trained model
model = unet()
model.load_weights("unet_membrane.keras")

# Path to test images
test_path = "data/membrane/test"

# Create test data generator
testGene = testGenerator(test_path)

# Predict
num_test_images = len(os.listdir(test_path))
results = model.predict(testGene, steps=num_test_images, verbose=1)

# Convert results to 8-bit before saving (avoids 'mode F' PNG error)
results_uint8 = [img_as_ubyte(img) for img in results]

# Save results
saveResult(test_path, results_uint8)



/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 98 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [49]:
from data import trainGenerator, testGenerator
from model import unet


train_path = 'data/membrane/train'
test_path = 'data/membrane/test'

myGene = trainGenerator(batch_size=1, train_path=train_path,
                        image_folder='image', mask_folder='label',
                        aug_dict={}, target_size=(64,64))

testGene = testGenerator(test_path, target_size=(64, 64))
model = unet(input_size=(64,64,1))

In [47]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('unet_membrane.keras', monitor='loss', verbose=1, save_best_only=True)

model.fit(myGene, steps_per_epoch=30, epochs=1, callbacks=[model_checkpoint])

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7289 - loss: 0.5763
Epoch 1: loss improved from inf to 0.54068, saving model to unet_membrane.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 23s 167ms/step - accuracy: 0.7301 - loss: 0.5751


In [50]:
results = []
for i in range(num_test_images):
    img = next(testGene)
    pred = model.predict(img)  # shape: (1,64,64,1)
    results.append(pred[0])


/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_78
Received: inputs=('Tensor(shape=(1, 64, 64, 1))',)
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━

StopIteration: 

In [51]:
!python main.py

2025-08-30 19:01:45.500112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756580505.520528   11686 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756580505.526851   11686 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756580505.542659   11686 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756580505.542693   11686 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756580505.542699   11686 computation_placer.cc:177] computation placer alr

In [53]:
from model import *
from data import *
from tensorflow.keras.callbacks import ModelCheckpoint # Import ModelCheckpoint

data_gen_args = dict(rotation_range=0.2, width_shift_range=0.05, height_shift_range=0.05, shear_range=0.05, zoom_range=0.05, horizontal_flip=True, fill_mode='nearest')

myGene = trainGenerator(20, 'data/membrane/train', 'image', 'label', data_gen_args, save_to_dir = None)

model = unet()
model_checkpoint = ModelCheckpoint('unet_salt.keras', monitor='loss', verbose=1, save_best_only=True) # Changed extension to .keras

model.fit(myGene, steps_per_epoch=10, epochs=5, callbacks=[model_checkpoint]) # Changed fit_generator to fit

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.7602 - loss: 0.5403
Epoch 1: loss improved from inf to 0.50770, saving model to unet_salt.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 199s 9s/step - accuracy: 0.7612 - loss: 0.5373
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 930ms/step - accuracy: 0.7896 - loss: 0.4293
Epoch 2: loss improved from 0.50770 to 0.42098, saving model to unet_salt.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.7899 - loss: 0.4285  
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 948ms/step - accuracy: 0.8017 - loss: 0.3847
Epoch 3: loss improved from 0.42098 to 0.37548, saving model to unet_salt.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.8024 - loss: 0.3839  
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 942ms/step - accuracy: 0.8336 - loss: 0.3458
Epoch 4: loss improved from 0.37548 to 0.34488, saving model to unet_salt.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - ac

In [59]:
from data import testGenerator, saveResult
from model import unet
import os
from skimage import img_as_ubyte # Import img_as_ubyte

gene = testGenerator('data/membrane/test', target_size=(64, 64))
unet = unet(input_size=(64,64,1))
unet.load_weights('unet_salt.keras')
pred = unet.predict(gene, steps=len(os.listdir('data/membrane/test')), verbose=1)

# Convert predictions to 8-bit unsigned integer format and split into a list of individual images
pred_uint8 = img_as_ubyte(pred)
pred_list = [pred_uint8[i] for i in range(pred_uint8.shape[0])] # Split the batch into a list of images

saveResult("data/membrane/test", pred_list) # Pass the list of images to saveResult

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 98 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


Problem 2

# U-Net: Simple Explanation

**Reference:**  
O. Ronneberger, P. Fischer, T. Brox. *U-Net: Convolutional Networks for Biomedical Image Segmentation.* MICCAI 2015.



## What U-Net Does
- U-Net is a **neural network for image segmentation**.  
- Segmentation means labeling each pixel in an image (e.g., which pixels belong to a cell).  
- It works especially well for **biomedical images** where datasets are small.



## How U-Net Works

### 1. Encoder (Contracting Path)
- Think of this as the network **looking at the big picture**.  
- Uses **convolutions + pooling** to shrink the image but keep important features.  

### 2. Bottleneck
- The **smallest, most compressed version** of the image features.  
- Acts as a bridge between understanding the big picture and rebuilding the image.

### 3. Decoder (Expanding Path)
- **Upsamples** the features to the original image size.  
- Uses **skip connections** from the encoder to keep details.  
- Combines **high-level ideas** (from encoder) with **low-level details** for precise segmentation.



## Key Ideas Made Simple
- **Skip connections**: help the network remember details while learning global context.  
- **Works with small data**: uses **data augmentation** like rotation and elastic deformations.  
- **End-to-end training**: network learns directly from images to pixel labels.  



## Why It’s Useful
- Segments images **accurately**, even with few training examples.  
- Fast and efficient, so practical for real biomedical applications.  
- Forms the base for many newer segmentation models like **3D U-Net** or **Attention U-Net**.


## Overall
This paper presents a network and training strategy that relies on the strong use of data augmentation to use the available annotated samples more efficiently.

This network is based on a fully convolutional network, It also uses many layers and in this network pooling operations are replaced with upsampling operators.